# Clase 43

Para una mejor visualización entrar al siguiente [link](https://nbviewer.jupyter.org/github/racsosabe/Miscelanea/blob/master/UPC/Clase%2043%20-%20Estructuras%20de%20Datos%20VIII.ipynb)

# Requisitos Previos

* Heavy-Light Decomposition
* Segment Tree

# Extendiendo el HLD: Small to Large Trick

Con Heavy-Light Decomposition consideramos asignar tipos a las aristas, definiendo dichos tipos para que se cumplan ciertas condiciones sobre el camino desde la raiz de un árbol hasta cada nodo $u$.

Consideremos ahora el caso en el que se nos da un árbol $T$, sobre el cual nosotros queremos responder a cierta información de cada uno de sus nodos $u$, considerando que dicha información depende del subárbol de $u$.

Es importante recordar la definición de arista pesada y ligera, pues debido a esta se da que existen una cantidad de $O(\log{n})$ aristas ligeras en el camino desde la raíz del árbol hasta cualquier nodo $u$.

Entonces, si pudiéramos procesar cada nodo dicha cantidad de aristas ligeras y una vez por cada cadena pesada, tendríamos una complejidad de $O(n\log{n})$.

Es ahora que podemos plantear la siguiente idea:

Si procesamos con un DFS cada nodo, considerando que no borraremos la información del subárbol al cual accedemos mediante una arista pesada y aplicamos el mismo criterio en la recursión, el trabajo hecho sobre cada nodo es $O(\log{n})$.

Para que sea más sencillo de ver, propondremos el pseudocódigo:

```C++
solve(u, keep):
    bigChild = -1
    for v in G[u]:
        if v is pi[u]: continue
        if (u, v) is light-edge:
            solve(v, false) // Resolvemos cada hijo de arista ligera pero sin guardar la información
        else:
            bigChild = v
    if bigChild != -1:
        solve(bigChild, true) // Resolvemos el hijo de arista pesada guardando la información
    addInfo(u) // Agregamos la información de todos los hijos de arista ligera y u
    // Resolver el nodo u con la información disponible
    if not keep:
        removeInfo(u) // Quitamos la información de todos los hijos de u y u
```

Si analizamos el proceso, es fácil notar que:

1) Si bajamos por una arista ligera, la información es procesada 1 vez y borrada luego de ello.
2) Si bajamos por una arista pesada, la información es procesada 1 vez y se mantiene.

Debido a lo anterior, como borramos la información del nodo $u$ por cada arista ligera, tendremos que reponerla la misma cantidad de veces, en total procesaremos la información del nodo $u$ $O(\log{n})$ veces. Finalmente, la complejidad será de $O(n\log{n})$ inserciones y eliminaciones de información.

## Implementación

Se puede plantear la implementación del pseudocódigo en base al preprocesamiento del Heavy-Light Decomposition que ya tenemos, con el cual colocábamos al hijo pesado en la primera posición de la lista de adyacencia y eliminábamos al padre.

```C++
void DFS(int u, int p = -1){
    subtree[u] = 1;
    for(int i = 0; i < G[u].size(); i++){
        if(G[u][i] == p){
            swap(G[u].back(), G[u][i]);
        }
        int v = G[u][i];
        if(v == p) continue;
        DFS(v, u);
        if(subtree[v] > subtree[G[u][0]]){
            swap(G[u][i], G[u][0]);
        }
        subtree[u] += subtree[v];
    }
    if(p != -1){
        G[u].pop_back();
    }
}


void add(int u, int val, int start){
    if(val == 1) agregarInfo(u);
    else quitarInfo(u);
    for(int i = start; i < G[u].size(); i++){
        add(G[u][i], val, 0); // Solo debemos restringir en el primer nivel
    }
}

void solve(int u, int keep){
    for(int i = 1; i < G[u].size(); i++){
        solve(G[u][i], 0);
    }
    if(!G[u].empty()) solve(G[u][0], 1);
    add(u, 1, 1);
    // Resolver con la información disponible
    if(!keep){
        add(u, -1, 0);
    }
}
```

Aprovechando la naturaleza de la lista de adyacencia, planteamos la función `add` con una variable $start$ que define si ignoramos o consideramos el hijo pesado, de manera que solo debemos restringir las aristas del primer nivel de descendientes (hijos directos), así que en la recursión deberemos llamar para cada hijo con $start = 0$.

## Problemas para practicar

- [Summing in a Tree](https://www.hackerrank.com/contests/101hack47/challenges/summing-in-a-tree)
- [Lomsat gelral](https://codeforces.com/contest/600/problem/E)
- [Tree Requests](https://codeforces.com/contest/570/problem/D)
- [Tree and Queries](https://codeforces.com/problemset/problem/375/D)